# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from config import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,1.76,67,96,11.48,TF,1698625920
1,1,adamstown,-25.0660,-130.1015,22.52,82,100,4.70,PN,1698625920
2,2,san antonio de pale,-1.4014,5.6325,26.33,83,55,3.78,GQ,1698625920
3,3,hamamatsu,34.7000,137.7333,18.90,57,0,0.45,JP,1698625714
4,4,hammerfest,70.6634,23.6821,-1.75,80,75,1.54,NO,1698625921


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# %%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "City"
)

# Display the map
map_plot_1

c:\Users\ranil\anaconda3\envs\dev\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df.loc[(city_data_df["Max Temp"] <= 26) & (city_data_df["Max Temp"] >= 21) & (city_data_df["Wind Speed"] < 4.5) & (city_data_df["Cloudiness"] == 0)]
# print(ideal_weather_df.count())

# Drop any rows with null values
ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,5,lompoc,34.6391,-120.4579,23.16,43,0,4.02,US,1698625313
105,105,gebeit,21.0667,36.3167,23.31,82,0,2.74,SD,1698625947
161,161,awjilah,29.1081,21.2869,21.21,45,0,2.94,LY,1698625962
188,188,laguna,38.4210,-121.4238,23.99,16,0,3.60,US,1698625968
224,224,ilheus,-14.7889,-39.0494,22.69,81,0,2.47,BR,1698625977
232,232,pacific grove,36.6177,-121.9166,24.82,61,0,2.06,US,1698625884
304,304,asansol,23.6833,86.9833,21.35,71,0,1.83,IN,1698625997
314,314,harper,4.3750,-7.7169,25.51,88,0,2.59,LR,1698626000
334,334,lindsay,36.2030,-119.0882,21.11,30,0,4.12,US,1698626005
343,343,saucillo,28.0167,-105.2833,24.97,17,0,2.17,MX,1698626008


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
5,lompoc,US,34.6391,-120.4579,43,
105,gebeit,SD,21.0667,36.3167,82,
161,awjilah,LY,29.1081,21.2869,45,
188,laguna,US,38.4210,-121.4238,16,
224,ilheus,BR,-14.7889,-39.0494,81,
232,pacific grove,US,36.6177,-121.9166,61,
304,asansol,IN,23.6833,86.9833,71,
314,harper,LR,4.3750,-7.7169,88,
334,lindsay,US,36.2030,-119.0882,30,
343,saucillo,MX,28.0167,-105.2833,17,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey":geoapify_key,
    "format":"json",
}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    print(f"lat={lat},lon={lon}")

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
lat=34.6391,lon=-120.4579
lompoc - nearest hotel: Embassy Suites by Hilton Lompoc Central Coast
lat=21.0667,lon=36.3167
gebeit - nearest hotel: No hotel found
lat=29.1081,lon=21.2869
awjilah - nearest hotel: No hotel found
lat=38.421,lon=-121.4238
laguna - nearest hotel: Holiday Inn Express & Suites
lat=-14.7889,lon=-39.0494
ilheus - nearest hotel: No hotel found
lat=36.6177,lon=-121.9166
pacific grove - nearest hotel: Pacific Grove Inn
lat=23.6833,lon=86.9833
asansol - nearest hotel: No hotel found
lat=4.375,lon=-7.7169
harper - nearest hotel: No hotel found
lat=36.203,lon=-119.0882
lindsay - nearest hotel: No hotel found
lat=28.0167,lon=-105.2833
saucillo - nearest hotel: No hotel found
lat=-28.8667,lon=153.5667
ballina - nearest hotel: Ballina Beach Resort
lat=37.3138,lon=-121.0208
newman - nearest hotel: No hotel found
lat=30.1471,lon=-85.6146
bay harbor - nearest hotel: Comfort Suites
lat=-32.035,lon=-52.0986
rio grande - nearest hotel: Hotel Vila Moura Execu

,City,Country,Lat,Lng,Humidity,Hotel Name
5,lompoc,US,34.6391,-120.4579,43,Embassy Suites by Hilton Lompoc Central Coast
105,gebeit,SD,21.0667,36.3167,82,No hotel found
161,awjilah,LY,29.1081,21.2869,45,No hotel found
188,laguna,US,38.4210,-121.4238,16,Holiday Inn Express & Suites
224,ilheus,BR,-14.7889,-39.0494,81,No hotel found
232,pacific grove,US,36.6177,-121.9166,61,Pacific Grove Inn
304,asansol,IN,23.6833,86.9833,71,No hotel found
314,harper,LR,4.3750,-7.7169,88,No hotel found
334,lindsay,US,36.2030,-119.0882,30,No hotel found
343,saucillo,MX,28.0167,-105.2833,17,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
# %%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    color = "City",
    hover_cols = ["Country", "Hotel Name"]
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name)